In [1]:
import requests
import pandas as pd
import json
import re
import polars as pl
import Enum_data as ed
pl.Config.set_tbl_rows(1000)
from persiantools import characters, digits
import sqlite3
import general_functions as gf

In [2]:
def get_table(url: str, table: int):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Origin': 'https://www.codal.ir',
    'Connection': 'keep-alive',
    'Referer': 'https://www.codal.ir/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Cookie': 'TS018fb0f7=01f9930bd2e2675d04882f623c888052df60031e7775d8e3c459dc4cb96bf8e870e0a9bf0e0eedff68bcce23fd9701c7a7fc0855c4; Unknown=1076170924.20480.0000'
    }
    response = requests.request("GET", url)
    statement = response.text
    pattern = r"var datasource = (.*?});"
    match = re.search(pattern, statement)
    if match:
        text = match.group(1)
    records = []
    records.append(
        (statement, text))
    for _, data in records:
        continue
    items = json.loads(data)['sheets']

    if isinstance(table, list):
        cells = []
        for t in table:
            raw_cells = items[0]['tables'][t]['cells']
            cells.append([(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in raw_cells])
        return [x for xs in cells for x in xs]
    
    cells = items[0]['tables'][table]['cells']
    return [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]


In [3]:
def create_dict_dataframes(url: str, date: int, report_type: str) -> dict:
    all_data ={ 'report_this_year': pl.DataFrame(),
                'est_remain': pl.DataFrame(),
                'est_next_year': pl.DataFrame()}

    cells_tuples = get_table(url, ed.tabels[report_type].value)
    dates = sorted(list(set([i[-1] for i in cells_tuples if i[-1] != ''])))
    for date_ in dates:
        products = []
        filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == date_]
        products = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '']
        df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
        df = df.pivot(values="value", on="col", index="row").sort("row")
        
        df_products = pl.from_records(products, schema=["col", "row", "value"], orient="row")
        df_products = df_products.pivot(values="value", on="col", index="row").sort("row")

        df_products = df_products.filter(df_products['row']>=df['row'][0] ,df_products['row']<=df['row'][-1])
        df = df_products.join(df, on="row", how="left")

        for i, col in enumerate(df.columns):
            cols = ed.cols[report_type].value
            value = df[col][0]        # value from first row
            if(isinstance(value, str)):
                for val in value.split():
                    if'/' in val:
                        num = int(val.replace('/', ''))
                        next_cols = df.columns[i : i + cols + 1]
                        cols_to_select = ['1','2', *next_cols]
                        
                        if num == date :
                            all_data['report_this_year'] = df.select(cols_to_select)
                            all_data['report_this_year'] = all_data['report_this_year'].with_row_index("row")
                            all_data['report_this_year'] = all_data['report_this_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                        else:
                            rows = len(all_data['report_this_year']['row'])
                            if(val[:4] == str(date)[:4]):
                                est_df = df.select(cols_to_select)
                                est_df = est_df.with_row_index("row")
                                all_data['est_remain'] = est_df.head(rows)
                                all_data['est_remain'] = all_data['est_remain'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))                          
                                
                                all_data['est_next_year'] = est_df.tail(rows)
                                all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))

    return all_data


In [4]:
def creat_raw_material_dataframe(symbol: str, url: str, date: int, period: int, publish: int) -> dict:
    all_data = create_dict_dataframes(url, date, 'RawMaterial')

    for key, data in all_data.items():
               
        cols = data.columns
        sorted_cols = sorted(cols, key=lambda x: (0 if x == "row" else 1, int(x) if x.isdigit() else float("inf")))
        sorted_cols = ['row', 'Date', '1', '2', '3', '4', '6', '7', '9', '10', '12', '13']
        data = data.select(sorted_cols)

        col_name = {
            "Raw Material": '1',
            'Unit': '2',
            "Beginning Inventory_qn": '3',
            "Beginning Inventory_pr": '4',
            "Purchases During the Period_qn": '6',
            "Purchases During the Period_pr": '7',
            "Consumption_qn": '9',
            "Consumption_pr": '10',
            "Ending Inventory_qn": '12',
            "Ending Inventory_pr": '13'
        }
        
        data = data.with_columns(data["1"].map_elements(lambda x: characters.ar_to_fa(x)).alias('1'))
        idx_internal_raw = data.filter(data['1'] == "مواد اولیه داخلی:")['row'][0]
        idx_total_internal_raw= data.filter(data['1']== "جمع مواد اولیه داخلی")['row'][0]
        df_internal_raw = data.filter(data['row']>idx_internal_raw,data['row']<idx_total_internal_raw-1)
        df_internal_raw=df_internal_raw.with_columns(pl.lit("داخلی").alias("Type"))
        idx_export_raw = data.filter(data['1'] == "مواد اولیه وارداتی:")['row'][0]
        idx_total_export_raw= data.filter(data['1']== "جمع مواد اولیه وارداتی")['row'][0]
        df_export_raw = data.filter(data['row']>idx_export_raw,data['row']<idx_total_export_raw-1)
        df_export_raw=df_export_raw.with_columns(pl.lit("وارداتی").alias("Type"))
        df_goods = pl.concat([df_internal_raw,df_export_raw])

        rename_map = {v: k for k, v in col_name.items()}
        df_goods = df_goods.rename(rename_map)
        
        df_goods = df_goods.insert_column(0, pl.lit(symbol).alias("Symbol"))
        df_goods = df_goods.insert_column(2, pl.lit(period).alias("Period"))
        df_goods = df_goods.insert_column(3, pl.lit(publish).alias("Publish"))
        cs = ['Symbol','Period','Publish','Date', 'Raw Material', 'Unit', 'Type', 'Beginning Inventory_qn', 'Beginning Inventory_pr', 'Purchases During the Period_qn', 'Purchases During the Period_pr', 'Consumption_qn', 'Consumption_pr', 'Ending Inventory_qn', 'Ending Inventory_pr']
        df_goods = df_goods[cs]
        all_data[key] = df_goods
    
    return all_data     

In [5]:
def convert_df_db_format(df: pl.DataFrame, report_type: str) -> pl.DataFrame:

    conn=sqlite3.connect("codal.sqlite")
    names = ['Symbol', 'Product', 'Type', 'Unit']
    col_names = ['key','Symbol_id','Date','Period', 'Publish','value','Price']
    rows = conn.execute(f"SELECT name, id FROM Symbol").fetchall()
    lookup = {key: value for key, value in rows}
    df = df.with_columns(pl.col(f"Symbol").replace(lookup).alias(f"Symbol_id"))
    key_df = df.with_columns((pl.col("Date").cast(pl.Utf8)+ pl.col("Symbol_id").cast(pl.Utf8)).alias("key").cast(pl.Int64))
    key_df = key_df[col_names]
    key_df = key_df.with_columns(pl.all().cast(pl.Int64))
    return key_df

In [6]:
#---------------------------------------
symbol = "فخوز"
financial_year = 14031230
report_type = ed.names.RawMaterial.value
#----------------------------------------

sheet_num = ed.sheets[report_type].value
table = ed.tabels[report_type].value
reports = gf.get_results(symbol=symbol, parse_date=gf.parse_date_persian(financial_year), report_type=report_type,sheet_num=sheet_num)
#reports
date = 14030930
period = int(reports[date]['period'])
publish = int(reports[date]['publish'])
url = reports[date]['url']

In [7]:
all_data = creat_raw_material_dataframe(symbol, url, date, period, publish)
all_data['est_remain']

Symbol,Period,Publish,Date,Raw Material,Unit,Type,Beginning Inventory_qn,Beginning Inventory_pr,Purchases During the Period_qn,Purchases During the Period_pr,Consumption_qn,Consumption_pr,Ending Inventory_qn,Ending Inventory_pr
str,i32,i32,i32,str,str,str,str,str,str,str,str,str,str,str
"""فخوز""",9,14031030,14031230,"""سنگ آهن""","""تن""","""داخلی""","""844608""","""58235052""","""1965341""","""60266415""","""1376803""","""57109064""","""1433146""","""62102477"""
"""فخوز""",9,14031030,14031230,"""آهن قراضه و چدن""","""تن""","""داخلی""","""4811""","""163578674""","""20054""","""225936222""","""20062""","""220602034""","""4803""","""185755569"""
"""فخوز""",9,14031030,14031230,""" اسفنجی خریداری شده""","""تن""","""داخلی""","""0""","""0""","""14803""","""167820982""","""0""","""0""","""14803""","""167820982"""
"""فخوز""",9,14031030,14031230,"""فروآلیاژها""","""تن""","""داخلی""","""4795""","""855174557""","""19858""","""862156159""","""12929""","""829890401""","""11724""","""894882805"""
"""فخوز""",9,14031030,14031230,"""مواد افزودنی""","""تن""","""داخلی""","""12388""","""166424524""","""33119""","""155959147""","""33173""","""116849456""","""12334""","""271658100"""
"""فخوز""",9,14031030,14031230,"""گندله خریداری""","""تن""","""داخلی""","""630903""","""64251099""","""338311""","""76369592""","""854267""","""68481151""","""114947""","""68481100"""
"""فخوز""",9,14031030,14031230,"""سایرمواد""","""تن""","""داخلی""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
